In [14]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [15]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [16]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [17]:
!pip install torchdata

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [62]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

print(tokenizer("happy"))
print(vocab(tokenizer("happy")))
#print(vocab.__dir__())
print(vocab.get_itos()[2269])




def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)


#print(val_iter)
#print(val_data)
#print(val_data.shape)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

data, targets = get_batch(val_data, 0)
#print(data)
#print(data.shape)

for batch in data:
    print("=============one_batch==============")
    for word in batch:
        print(vocab.get_itos()[word.item()])
        #print(vocab.get_itos()[word.value])
        

['happy']
[2269]
happy
=============one_batch==============
=
to
michigan
support
.
the
states
37
having
which
=============one_batch==============
homarus
the
wolverines
to
=
following
to
m
been
happened
=============one_batch==============
gammarus
north
men
sweep
scientology
resolution
mark
)
a
during
=============one_batch==============
=
west
'
the
in
that
the
diameter
member
filming
=============one_batch==============
homarus
require
s
area
germany
,
game
french
of
.
=============one_batch==============
gammarus
considerable
basketball
in
=
having
'
burr
the
the
=============one_batch==============
,
care
team
front
the
heard
s
<unk>
unit
depiction
=============one_batch==============
known
.
=
of
church
all
fifteenth
is
,
of
=============one_batch==============
as
=
the
the
of
available
anniversary
located
or
the
=============one_batch==============
the
=
2011
marine
scientology
evidence
.
on
a
police
=============one_batch==============
european
=
–
lines
has
regarding
it
a
<u

In [20]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [34]:
ntokens = len(vocab)  # size of vocabulary
print(ntokens)
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

28782


In [63]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            print(output_flat)
            print(output_flat.shape)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)

    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 303.64 | loss  8.01 | ppl  2998.86
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 293.29 | loss  6.87 | ppl   965.37
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 299.40 | loss  6.44 | ppl   626.80
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 292.68 | loss  6.30 | ppl   546.28
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 283.52 | loss  6.18 | ppl   481.74
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 289.49 | loss  6.16 | ppl   471.18
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 283.60 | loss  6.11 | ppl   451.78
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 285.30 | loss  6.11 | ppl   448.27
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 287.96 | loss  6.02 | ppl   412.18
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 290.74 | loss  6.01 | ppl   408.31
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 292.77 | loss  5.89 | ppl   362.98
| epoch   

tensor([[ 8.3671,  5.9517,  9.0122,  ..., -1.2047, -1.0007, -1.1687],
        [ 8.4810,  8.8118,  9.5768,  ..., -0.1316, -0.8331, -0.3549],
        [ 7.5962,  7.4014, 11.6258,  ..., -0.6232, -0.6595, -0.6936],
        ...,
        [ 8.1101,  8.2493,  6.2740,  ..., -0.3153, -1.0494, -0.7307],
        [ 7.5472,  6.9440,  9.9867,  ..., -1.3368, -0.2986, -0.0926],
        [ 8.1636, 11.5466,  6.9560,  ..., -0.0435,  0.1705, -0.0894]])
torch.Size([350, 28782])
tensor([[ 7.7204,  6.7584,  8.0878,  ..., -1.0389, -0.3779, -0.8066],
        [ 8.4487,  7.6965, 11.3006,  ..., -0.5485, -0.3095, -0.6149],
        [ 8.1150,  6.5641, 11.3605,  ..., -0.6479, -0.6588, -0.6404],
        ...,
        [ 7.1069,  6.8911, 10.7383,  ..., -0.2214, -0.2006, -0.9144],
        [ 7.0118,  8.5613,  7.6263,  ..., -0.4698, -0.4073, -0.1555],
        [ 7.9561,  8.0052, 10.8175,  ..., -0.9577, -0.7969, -0.4855]])
torch.Size([350, 28782])
tensor([[ 8.7392,  8.7304,  5.5793,  ...,  0.2311,  0.0662, -0.5707],
        [ 7.

tensor([[ 8.6644,  7.5515,  9.8941,  ..., -0.4834, -0.8264, -0.5529],
        [ 8.0191,  7.2913, 11.0881,  ..., -0.4545, -0.3865, -0.3968],
        [ 8.4642,  7.4810,  6.6031,  ..., -0.6109, -0.2080, -0.4001],
        ...,
        [ 7.5815,  6.4185, 11.2902,  ..., -0.5893, -0.7215, -0.7306],
        [ 7.5810,  7.3901,  9.6073,  ..., -0.4409,  0.0417, -1.0424],
        [ 7.6313,  7.0055, 11.9468,  ..., -0.5609, -0.7746, -0.6817]])
torch.Size([350, 28782])
tensor([[ 8.7444,  9.3623,  7.0783,  ..., -0.4958, -0.2934, -0.7096],
        [ 8.1468,  7.8051,  6.8638,  ...,  0.1257, -0.4705, -0.2617],
        [ 8.4642,  7.4810,  6.6031,  ..., -0.6109, -0.2080, -0.4001],
        ...,
        [ 8.4461,  7.2955, 11.3881,  ..., -0.3247, -0.5323, -0.7603],
        [ 7.5521,  7.1614,  9.7299,  ..., -1.4019, -0.7372, -0.2424],
        [ 7.2892,  5.5435,  7.4645,  ..., -0.6321, -0.4781, -1.0412]])
torch.Size([350, 28782])
tensor([[ 6.9707,  4.3037,  4.9493,  ..., -0.7395, -0.1933, -0.4140],
        [ 6.

tensor([[ 8.7444,  9.3623,  7.0783,  ..., -0.4958, -0.2934, -0.7096],
        [ 8.9872,  6.9087, 11.3541,  ..., -0.3385, -0.7642, -0.5251],
        [ 8.4460,  9.7666,  7.5616,  ...,  0.2764,  0.6032,  0.0648],
        ...,
        [ 7.8739,  8.2687,  6.9698,  ..., -1.1194, -0.7233, -0.5181],
        [ 8.4526,  5.1292,  6.3676,  ..., -0.6010, -1.0151, -0.6912],
        [ 7.5542,  5.4140,  8.5012,  ..., -0.5732, -1.1291, -0.7408]])
torch.Size([350, 28782])
tensor([[ 8.2242,  6.9055, 11.2508,  ..., -0.4399, -0.8170, -0.5834],
        [ 8.1489,  6.6791, 11.1102,  ..., -0.5035, -0.8043, -0.6916],
        [ 8.7392,  8.7304,  5.5793,  ...,  0.2311,  0.0662, -0.5707],
        ...,
        [ 8.3146,  8.8795,  6.2004,  ..., -0.2773, -0.6341, -0.0446],
        [ 8.0962, 10.6509,  7.3017,  ..., -0.4084, -0.2448, -0.4843],
        [ 7.5446,  7.1494, 11.1395,  ..., -0.6592, -0.7403, -1.1291]])
torch.Size([350, 28782])
tensor([[ 8.4295,  6.2402, 10.4421,  ..., -0.6538, -0.6120, -0.6375],
        [ 7.

tensor([[ 8.1468,  7.8051,  6.8638,  ...,  0.1257, -0.4705, -0.2617],
        [ 6.9707,  4.3037,  4.9493,  ..., -0.7395, -0.1933, -0.4140],
        [ 8.0828,  7.2232, 11.4239,  ..., -0.5930, -0.7137, -0.5622],
        ...,
        [ 7.9991,  5.8283,  7.4842,  ..., -0.8207, -1.0086, -0.6280],
        [ 8.3561,  5.3508,  7.3961,  ..., -1.6124, -1.1273, -0.5542],
        [ 8.8638,  7.3595,  6.8448,  ..., -0.6297, -0.4515, -0.3992]])
torch.Size([350, 28782])
tensor([[ 6.9567,  7.3852, 11.8862,  ..., -0.7780, -0.3477, -0.5527],
        [ 8.2341,  8.9039,  8.4824,  ..., -0.1465, -0.4206, -0.3184],
        [ 7.9535,  7.0810, 11.7963,  ..., -0.6013, -0.7950, -0.5943],
        ...,
        [ 7.6152,  4.5529,  5.1285,  ..., -0.9075, -0.3143, -0.4670],
        [ 8.2168,  8.3008,  6.1178,  ..., -0.3899, -1.1543, -0.4861],
        [ 8.1139,  5.1778,  5.7311,  ..., -0.7029, -0.9052, -0.8031]])
torch.Size([350, 28782])
tensor([[ 7.2680,  8.9907, 10.1254,  ..., -0.5829,  0.2114,  0.1335],
        [ 7.

...

tensor([[ 9.0220,  7.8394,  9.0610,  ..., -0.2844, -0.2378, -0.6652],
        [ 7.5199,  6.3172,  7.2972,  ..., -0.2383, -0.2168, -0.9581],
        [ 7.9458,  7.1932, 11.6289,  ..., -0.5267, -0.6853, -0.6222],
        ...,
        [ 8.7131,  7.6631,  6.7261,  ...,  0.1076, -0.6044, -0.3982],
        [ 7.4629,  4.1847,  5.1855,  ..., -0.5710, -0.2756, -0.5406],
        [ 8.4440,  8.4975,  8.4984,  ...,  0.0625, -0.6736, -0.6572]])
torch.Size([350, 28782])
tensor([[ 8.9872,  6.9087, 11.3541,  ..., -0.3385, -0.7642, -0.5251],
        [ 8.2341,  8.9039,  8.4824,  ..., -0.1465, -0.4206, -0.3184],
        [ 7.3741,  9.4224,  7.8230,  ..., -0.2750, -0.2241, -0.6511],
        ...,
        [ 7.1954,  8.0933, 12.0189,  ..., -0.9339, -0.5290,  0.5293],
        [ 7.1723,  6.3330, 11.2924,  ..., -0.7247, -0.3833, -1.0465],
        [ 8.6140,  4.8308,  5.6889,  ..., -1.1942, -0.7518, -0.9639]])
torch.Size([350, 28782])
tensor([[ 8.0501e+00,  5.8516e+00,  6.4160e+00,  ..., -7.7186e-01,
         -6.889

tensor([[ 8.2592,  8.6983,  7.5406,  ..., -0.5877, -1.2140, -0.6451],
        [ 8.2341,  8.9039,  8.4824,  ..., -0.1465, -0.4206, -0.3184],
        [ 7.1710,  5.3916,  7.8624,  ..., -0.4701, -0.9755, -0.6534],
        ...,
        [ 8.6982,  6.9427, 11.6147,  ..., -0.4212, -0.7229, -0.6133],
        [ 7.1543,  5.8041,  7.7628,  ..., -0.6841, -0.2751, -1.2327],
        [ 8.4619,  7.7146,  6.9530,  ...,  0.1345, -0.5753, -0.6247]])
torch.Size([350, 28782])
tensor([[ 8.7444,  9.3623,  7.0783,  ..., -0.4958, -0.2934, -0.7096],
        [ 9.1620,  6.7315,  7.1244,  ...,  0.8948, -0.6346,  0.1802],
        [ 8.7392,  8.7304,  5.5793,  ...,  0.2311,  0.0662, -0.5707],
        ...,
        [ 9.1403,  7.3713,  6.7107,  ..., -0.5664, -0.5854, -0.4010],
        [ 8.1676,  7.3949, 11.5146,  ..., -0.6307, -0.5417, -0.7045],
        [ 7.7713,  6.5237, 11.1730,  ..., -1.1861, -0.4933, -0.7295]])
torch.Size([350, 28782])
tensor([[ 8.0501,  5.8516,  6.4160,  ..., -0.7719, -0.6889, -0.6730],
        [ 8.

tensor([[ 7.2424,  7.6062,  9.4396,  ..., -0.6969, -0.0945, -1.2026],
        [ 7.9933,  7.2730, 11.5974,  ..., -0.4479, -0.6561, -0.5196],
        [ 7.6933,  7.2647, 11.2187,  ..., -0.5062, -0.5656, -0.5055],
        ...,
        [ 7.5724,  6.4607,  9.8694,  ..., -1.1219, -0.1410,  0.1752],
        [ 8.1002,  6.8843, 10.7600,  ..., -0.8761, -0.2654, -0.8930],
        [ 8.0962,  8.1303,  8.4490,  ...,  0.2722, -0.6630, -0.7809]])
torch.Size([350, 28782])
tensor([[ 7.2138,  8.2846,  7.5789,  ..., -0.6269, -0.2163, -0.4475],
        [ 7.8807,  6.9564, 10.0751,  ..., -1.3341, -0.9204, -0.7858],
        [ 8.7392,  8.7304,  5.5793,  ...,  0.2311,  0.0662, -0.5707],
        ...,
        [ 7.4836,  7.6829, 13.0906,  ..., -1.0104, -0.3775, -0.6887],
        [ 8.1036,  5.1894,  5.7963,  ..., -0.8566, -0.8347, -0.7613],
        [ 7.5882,  8.4833,  7.2809,  ...,  0.0190, -0.5035, -0.5247]])
torch.Size([350, 28782])
tensor([[ 8.1826,  7.0976, 11.4325,  ..., -0.5075, -0.8075, -0.6287],
        [ 8.

tensor([[ 7.2424,  7.6062,  9.4396,  ..., -0.6969, -0.0945, -1.2026],
        [ 8.0945, 10.6572,  7.3312,  ...,  0.2458, -0.1983, -0.2541],
        [ 8.2842,  9.6790,  6.3743,  ..., -0.1674, -0.2976, -0.3849],
        ...,
        [ 7.4699,  7.0413, 11.2614,  ..., -0.5385, -0.5645, -0.9576],
        [ 7.5816,  8.7515, 11.5275,  ..., -0.3430, -0.5362, -0.3084],
        [ 7.2630,  8.4777,  7.9760,  ..., -0.3797, -0.4786, -0.4224]])
torch.Size([350, 28782])
tensor([[ 8.7392,  8.7304,  5.5793,  ...,  0.2311,  0.0662, -0.5707],
        [ 8.2155,  8.2186,  7.0232,  ..., -0.5769, -1.1858, -0.6381],
        [ 7.6482,  7.2145, 11.8503,  ..., -0.7539, -0.6609, -0.6955],
        ...,
        [ 7.6173,  6.5591,  9.3483,  ..., -1.3179, -0.2348, -0.1390],
        [ 7.9002,  8.5797,  6.8144,  ..., -0.7759, -0.8077, -0.5173],
        [ 7.2157,  8.1959,  8.1162,  ..., -0.2353, -0.3015, -0.3775]])
torch.Size([350, 28782])
tensor([[ 8.2341,  8.9039,  8.4824,  ..., -0.1465, -0.4206, -0.3184],
        [ 6.

| epoch   2 |  1400/ 2928 batches | lr 4.75 | ms/batch 312.94 | loss  5.69 | ppl   295.30
| epoch   2 |  1600/ 2928 batches | lr 4.75 | ms/batch 329.81 | loss  5.70 | ppl   299.66
| epoch   2 |  1800/ 2928 batches | lr 4.75 | ms/batch 330.91 | loss  5.64 | ppl   282.65


In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)